In [1]:
from urllib.request import urlopen
from IPython.display import clear_output
import socket
import urllib
import json
import csv

group_name = "formula1fc"
timeout = 5
count = 100
# output = "post_data.txt"

def receive_data(request):
    received = False
    data = None
    while not received:
        try:
            data = urllib.request.urlopen(request, 
                timeout = timeout).read().decode("utf-8")
        except (urllib.error.HTTPError, urllib.error.URLError) as error:
            print("Data was not received")
        except socket.timeout:
            print("socket timed out")
        else:
            received = True
    return data


def get_group_info(group_name):

    infojson = receive_data("https://api.vk.com/method/groups.getById?v=5.60"
        "&group_id=" + group_name)
    obj = json.loads(infojson)
    return obj["response"][0]

def get_group_size(group_id):
    sizejson = receive_data("https://api.vk.com/method/wall.get?v=5.60"
                "&owner_id=" + str(-group_id))

    obj = json.loads(sizejson)
    return obj["response"]["count"]

def get_group_quotes(group_id, count = 10, offset = 0, all_dict = None):
    quotepagejson = receive_data("https://api.vk.com/method/wall.get?v=5.60"
                "&owner_id=" + str(-group_id) + "&offset=" + str(offset) +
                "&count=" + str(count))

    obj = json.loads(quotepagejson)
    if "response" not in obj.keys():
        return []
    response = obj["response"]
    if "items" not in response.keys():
        return []

    answer = []
    for post in response["items"]:
        postid = None
        fromid = None
        date = None
        text = None
        comments = None
        likes = None
        typestr = None
        reposts = None

        if "id" in post.keys():
            postid = post["id"]
            if not all_dict is None:
                all_dict[postid] = post
        if "from_id" in post.keys():
            fromid = post["from_id"]
        if "date" in post.keys():
            date = post["date"]
        if "text" in post.keys():
            text = post["text"]
        if "type" in post.keys():
            typestr = post["post_type"]
        if "comments" in post.keys() and "count" in post["comments"].keys():
            comments = post["comments"]["count"]
        if "likes" in post.keys() and "count" in post["likes"].keys():
            likes = post["likes"]["count"]
        if "reposts" in post.keys() and "count" in post["reposts"].keys():
            reposts = post["reposts"]["count"]
        answer.append([postid, fromid, typestr, date, text, comments, likes, reposts])

    return answer

In [2]:
info = get_group_info(group_name)
print("Got group info")
print("id: {0}".format(info["id"]))
print("caption: {0}".format(info["name"]))

# size = get_group_size(info["id"])
count = 100
size = 7000
print("size: {0}".format(size))

currentoffset = 0
# csvfile = open(output, "w")
# csvobj = csv.writer(csvfile)
# csvobj.writerow(["id", "from_id", "type", "date", "text", "comments", "likes", "reposts"])
posts_dict = {}
while currentoffset < size:
    print("Current offset:{0}/{1}".format(currentoffset, size))
    newquotes = get_group_quotes(info["id"], count, currentoffset, posts_dict)
#     for i in newquotes:
#         csvobj.writerow(i)
#     csvfile.flush()
    currentoffset = currentoffset + len(newquotes)
    clear_output()

print("posts wrote: {0}".format(currentoffset))
print("done")

# csvfile.close()


posts wrote: 7000
done


In [3]:
import datetime
# posts_dict_copy = posts_dict
for post_raw_dict in posts_dict.values():
    if "date" in post_raw_dict.keys():
        post_raw_dict["date_py"] = datetime.datetime.fromtimestamp(post_raw_dict["date"])
        
for i in list(posts_dict.keys()):
    if not "date_py" in posts_dict[i] or posts_dict[i]["date_py"].year != 2016:
        del posts_dict[i]



In [4]:
def get_comments(group_id, post_id, dict_to, count = 100, offset = 0):
    quotepagejson = receive_data("https://api.vk.com/method/wall.getComments?v=5.60"
                "&owner_id=" + str(-group_id) + "&post_id=" + str(post_id) + "&offset=" + str(offset) +
                "&count=" + str(count) + "&need_likes=1")

    obj = json.loads(quotepagejson)
    if "response" not in obj.keys():
        return 0
    response = obj["response"]
    if "items" not in response.keys():
        return 0
    
    amount = 0
    
    for comment in response["items"]:
        key = (post_id, comment["id"])
        dict_to[key] = comment
        amount +=1

    return amount

comment_dict = {}
all_posts_size = len(posts_dict)
for num, i in enumerate(posts_dict.values()):
    currentoffset = 0
    size = 0
    if "comments" in i.keys():
        size = i["comments"]["count"]
    print("Fetching comments:{0}/{1}".format(num, all_posts_size))
    while currentoffset < size:
        print("Current offset:", currentoffset)
        addlen = get_comments(info["id"], i["id"], comment_dict, offset = currentoffset)
        currentoffset = currentoffset + addlen
    clear_output()


In [8]:
for comment_raw_dict in comment_dict.values():
    if "date" in comment_raw_dict.keys():
        comment_raw_dict["date_py"] = datetime.datetime.fromtimestamp(comment_raw_dict["date"])

In [87]:
#flood plot
import matplotlib.pyplot as plt
import matplotlib
flood_dict = {}
for post_raw_dict in posts_dict.values():
    counter = 1
#     if "comments" in post_raw_dict.keys() and "count" in post_raw_dict["comments"].keys():
#             counter += post_raw_dict["comments"]["count"]
    cur_key = (post_raw_dict["date_py"].month, post_raw_dict["date_py"].day)
    if cur_key in flood_dict.keys():
        flood_dict[cur_key] += counter
    else:
        flood_dict[cur_key] = counter
all_dates = sorted(list(flood_dict.keys()))
all_posts = [flood_dict[i] for i in all_dates]
plt.plot([datetime.date(2016, *i) for i in all_dates], all_posts)
hfont = {'fontname':'Arial'}
plt.title('Активность нашей группы в течение года', **hfont)
plt.ylabel('Суммарное число постов и комментариев к ним за день', **hfont)
plt.xlabel('Дата', **hfont)
matplotlib.rc('font', family='Arial')

plt.grid(True)
funnydates = [{"caption":"Австралия", "tupdate":(3, 20), "vertoff": 150},
             {"caption":"Бахрейн", "tupdate":(4, 3), "vertoff": 150},
             {"caption":"Китай", "tupdate":(4, 17), "vertoff": 100},
             {"caption":"Россия", "tupdate":(5, 1), "vertoff": 120},
             {"caption":"Испания", "tupdate":(5, 15), "vertoff": 150},
             {"caption":"Монако", "tupdate":(5, 29), "vertoff": 120},
             {"caption":"Канада", "tupdate":(6, 12), "vertoff": 120},
             {"caption":"Европа", "tupdate":(6, 19), "vertoff": 120},
             {"caption":"Австрия", "tupdate":(7, 3), "vertoff": 150},
             {"caption":"Великобритания", "tupdate":(7, 10), "vertoff": 250},
             {"caption":"Венгрия", "tupdate":(7, 24), "vertoff": 150},
             {"caption":"Германия", "tupdate":(7, 31), "vertoff": 150},
             {"caption":"Бельгия", "tupdate":(8, 28), "vertoff": 150},
             {"caption":"Италия", "tupdate":(9, 4), "vertoff": 120},
             {"caption":"Сингапур", "tupdate":(9, 18), "vertoff": 150},
             {"caption":"Малайзия", "tupdate":(10, 2), "vertoff": 150},
             {"caption":"Япония", "tupdate":(10, 9), "vertoff": 120},
             {"caption":"США", "tupdate":(10, 23), "vertoff": 75},
             {"caption":"Мексика", "tupdate":(10, 30), "vertoff": 150},
             {"caption":"Бразилия", "tupdate":(11, 13), "vertoff": 150},
             {"caption":"Абу-Даби", "tupdate":(11, 27), "vertoff": 150},
             {"caption":"Уход Росберга", "tupdate":(12, 2), "vertoff": 200},
             {"caption":"Рокировка Квят-Ферстаппен", "tupdate":(5, 5), "vertoff": 400},
             {"caption":"Тесты 22 февраля", "tupdate":(2, 22), "vertoff": 250}]
for i in funnydates:
    plt.annotate(i["caption"], xy=(datetime.date(2016, *i["tupdate"]), flood_dict[i["tupdate"]] + 
                                   i["vertoff"]), rotation="vertical", ha = "center")


plt.show()

In [5]:
len(posts_dict)

6712

In [6]:
html_beginning = """<!DOCTYPE html>
<html>
  <head>
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
    <!--<meta http-equiv="X-UA-Compatible" content="IE=edge; charset=utf-8">-->
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <meta name="description" content="">
    <meta name="author" content="">

    <title>{0}</title>

  </head>

  <body>
    <style type="text/css">

    table {{
    border-collapse: collapse;
    border-spacing: 1px 1px;

}}

table, th, td {{
    border: 1px solid black;
    text-align: center;
    vertical-align: middle;
}}

tr:nth-child(even) {{background-color: #f2f2f2}}

th {{
    background-color: #4CAF50;
    color: white;
}}

    </style>
"""

html_ending = """ </body>
</html>
"""


In [7]:
def get_certain_users(users):
    quotepagejson = receive_data("https://api.vk.com/method/users.get?v=5.60"
                "&user_ids=" + ",".join([str(i) for i in users]) + 
                "&fields=photo_50,city")

    obj = json.loads(quotepagejson)
    if "response" not in obj.keys():
        return []
    response = obj["response"]

    answer = {}
    for user in response:
        if "id" in user.keys():
            userid = user["id"]
            answer[userid] = user

    return answer

In [11]:
month_table_raw = {}
import pandas as pd
for post_raw_dict in posts_dict.values():
    cur_month = post_raw_dict["date_py"].month
    cur_key = post_raw_dict["from_id"]
    if cur_key == -info["id"]:
        continue
    if not cur_month in month_table_raw.keys():
        month_table_raw[cur_month] = {}
    if cur_key in month_table_raw[cur_month].keys():
        month_table_raw[cur_month][cur_key] += 1
    else:
        month_table_raw[cur_month][cur_key] = 1

month_table_dict = month_table_raw.copy()
        
for month in list(month_table_raw.keys()):
    month_table_raw[month] = sorted([(i, j) for i, j in month_table_raw[month].items()],
                                   key = lambda x : x[1], reverse = True)
    
points = [25, 18, 15, 12, 10, 8, 6, 4, 2, 1]

def give_points(standings, points):
    if len(standings) == 0:
        return standings
    result = [(i[0], 0) for i in standings]
    result[0] = (standings[0][0], points[0])
    for i in range(1, len(standings)):
        if standings[i][1] == standings[i - 1][1]:
            result[i] = (standings[i][0], result[i - 1][1])
        else:
            if i <= len(points) - 1:
                result[i] = (standings[i][0], points[i])
            else:
                break
    result = [i for i in result if i[1] > 0]
    return result

points_table = {i:give_points(month_table_raw[i], points) for i in month_table_raw.keys()}

final_table = {}
for i in points_table.values():
    for j in i:
        if j[0] in final_table.keys():
            final_table[j[0]] += j[1]
        else:
            final_table[j[0]] = j[1]
final_table = sorted([(i, j) for i, j in final_table.items()],
                     key = lambda x : x[1], reverse = True)
places = give_points(final_table, list(range(1, len(final_table) + 1)))

def get_month_dude_point(month, dude):
    for i in points_table[month]:
        if i[0] == dude:
            return "<b>{0}</b> ({1})".format(str(i[1]), str(month_table_dict[month][dude]))
    
    if dude in month_table_dict[month].keys():
        return "0 ({0})".format(str(month_table_dict[month][dude]))

    return "0 (0)"

dudes_info = get_certain_users([i[0] for i in final_table])

final_table_df = {"{:02}".format(i):{j[0]: get_month_dude_point(i, j[0]) for j in final_table} 
                  for i in range(1, 12 + 1)}
final_table_df["Place"] = {i[0]:"<b>{0}</b>".format(i[1]) for i in places}
final_table_df["place_int"] = {i[0]:i[1] for i in places}
final_table_df["Sum"] = {i[0]:"<b>{0}</b>".format(i[1]) for i in final_table}
final_table_df["Name"] = {i[0]:"<a class = 'dude-link' href = 'https://vk.com/id{0}'>{1} {2}</a>".format(
    i[0], dudes_info[i[0]]["first_name"], dudes_info[i[0]]["last_name"]) for i in final_table}
final_table_df["Photo"] = {i[0]:"<img class = 'dude-img' src = '{0}'>".format(
    dudes_info[i[0]]["photo_50"]) for i in final_table}

pd.set_option('display.max_colwidth', -1)
output = pd.DataFrame.from_dict(final_table_df).sort_values("place_int").to_html(escape = False,
    columns = ["Place", "Photo", "Name"] + ["{:02}".format(i) for i in range(1, 12+1)] + ["Sum"],
                                                                            index = False)
pd.set_option('display.max_colwidth', 20)
htmlfile = open("posts.html", "w", encoding = "utf-8")
htmlfile.write(html_beginning.format("Чемпионат топикстартеров"))
htmlfile.write("<h3>Личный зачёт Топикстартеров-2016</h3>")
htmlfile.write(output)
htmlfile.write(html_ending)
htmlfile.close()

# pd.DataFrame.from_dict(final_table_df).sort_values("place")

In [67]:
post_types = {}
for post_raw_dict in posts_dict.values():
    if "attachments" in post_raw_dict.keys():
        for i in post_raw_dict["attachments"]:
            post_types[i["type"]] = i[i["type"]]
# post_types
post_raw_dict

{'attachments': [{'link': {'caption': 'www.periscope.tv',
    'description': 'Toro Rosso (@ToroRossoSpy) from Faenza, EMR, Italy. Official Periscope account of   Scuderia Toro Rosso                  #GOTOROROSSO',
    'title': 'Toro Rosso on Periscope: "Max Verstappen at the factory in Faenza! #AskMax your questions ;)"',
    'url': 'https://www.periscope.tv/w/aYtnsjF4TmphZEx4ZXlLYmR8MWxQS3FQRWtMUFl4Yom90tMrdCbm0kDA_qKthGRbnXnmwGjafpfk5iWJNwRX'},
   'type': 'link'}],
 'comments': {'count': 1},
 'date': 1455124594,
 'date_py': datetime.datetime(2016, 2, 10, 20, 16, 34),
 'from_id': 171010528,
 'id': 537941,
 'likes': {'count': 1},
 'owner_id': -647,
 'post_type': 'post',
 'reposts': {'count': 0},
 'text': 'Максимка в перископе\nhttps://www.periscope.tv/w/aYtnsjF4TmphZEx4ZXlLYmR8MWxQS3FQRWtMUFl4Yom90tMrdCbm0kDA_qKthGRbnXnmwGjafpfk5iWJNwRX'}

In [196]:
month_table_raw = {}

max_vote = 0
max_vote_id = None

for post_raw_dict in posts_dict.values():
    votes = None
    if "attachments" in post_raw_dict.keys():
        for i in post_raw_dict["attachments"]:
            if i["type"] == "poll":
                votes = i["poll"]["votes"]
    if votes is None:
        continue
    cur_month = post_raw_dict["date_py"].month
    cur_key = post_raw_dict["from_id"]
    if cur_key == -info["id"]:
        continue
    if votes > max_vote:
        max_vote = votes
        max_vote_id = post_raw_dict["id"]
    if not cur_month in month_table_raw.keys():
        month_table_raw[cur_month] = {}
    if cur_key in month_table_raw[cur_month].keys():
        month_table_raw[cur_month][cur_key] += votes
    else:
        month_table_raw[cur_month][cur_key] = votes

month_table_dict = month_table_raw.copy()
        
for month in list(month_table_raw.keys()):
    month_table_raw[month] = sorted([(i, j) for i, j in month_table_raw[month].items()],
                                   key = lambda x : x[1], reverse = True)

points = [25, 18, 15, 12, 10, 8, 6, 4, 2, 1]

def give_points(standings, points):
    if len(standings) == 0:
        return standings
    result = [(i[0], 0) for i in standings]
    result[0] = (standings[0][0], points[0])
    for i in range(1, len(standings)):
        if standings[i][1] == standings[i - 1][1]:
            result[i] = (standings[i][0], result[i - 1][1])
        else:
            if i <= len(points) - 1:
                result[i] = (standings[i][0], points[i])
            else:
                break
    result = [i for i in result if i[1] > 0]
    return result

points_table = {i:give_points(month_table_raw[i], points) for i in month_table_raw.keys()}

final_table = {}
for i in points_table.values():
    for j in i:
        if j[0] in final_table.keys():
            final_table[j[0]] += j[1]
        else:
            final_table[j[0]] = j[1]
final_table = sorted([(i, j) for i, j in final_table.items()],
                     key = lambda x : x[1], reverse = True)
places = give_points(final_table, list(range(1, len(final_table) + 1)))

def get_month_dude_point(month, dude):
    for i in points_table[month]:
        if i[0] == dude:
            return "<b>{0}</b> ({1})".format(str(i[1]), str(month_table_dict[month][dude]))
    
    if dude in month_table_dict[month].keys():
        return "0 ({0})".format(str(month_table_dict[month][dude]))

    return "0 (0)"

dudes_info = get_certain_users([i[0] for i in final_table])

final_table_df = {"{:02}".format(i):{j[0]: get_month_dude_point(i, j[0]) for j in final_table} 
                  for i in range(1, 12 + 1)}
final_table_df["Place"] = {i[0]:"<b>{0}</b>".format(i[1]) for i in places}
final_table_df["place_int"] = {i[0]:i[1] for i in places}
final_table_df["Sum"] = {i[0]:"<b>{0}</b>".format(i[1]) for i in final_table}
final_table_df["Name"] = {i[0]:"<a class = 'dude-link' href = 'https://vk.com/id{0}'>{1} {2}</a>".format(
    i[0], dudes_info[i[0]]["first_name"], dudes_info[i[0]]["last_name"]) for i in final_table}
final_table_df["Photo"] = {i[0]:"<img class = 'dude-img' src = '{0}'>".format(
    dudes_info[i[0]]["photo_50"]) for i in final_table}

pd.set_option('display.max_colwidth', -1)
output = pd.DataFrame.from_dict(final_table_df).sort_values("place_int").to_html(escape = False,
    columns = ["Place", "Photo", "Name"] + ["{:02}".format(i) for i in range(1, 12+1)] + ["Sum"],
                                                                            index = False)
pd.set_option('display.max_colwidth', 20)
htmlfile = open("polls.html", "w")
htmlfile.write(html_beginning.format("Чемпионат социологов"))
htmlfile.write("<h3>Личный зачёт Социологов-2016</h3>")
htmlfile.write(output)
htmlfile.write(html_ending)
htmlfile.close()

print(max_vote)
print(max_vote_id)

554
615487


In [197]:
month_table_raw = {}
for post_raw_dict in posts_dict.values():
    count = None
    if "comments" in post_raw_dict.keys():
        count = post_raw_dict["comments"]["count"]
    if count is None:
        continue
    cur_month = post_raw_dict["date_py"].month
    cur_key = post_raw_dict["from_id"]
    if cur_key == -info["id"]:
        continue
    cur_key = (post_raw_dict["from_id"], post_raw_dict["id"])
    if not cur_month in month_table_raw.keys():
        month_table_raw[cur_month] = {}
    month_table_raw[cur_month][cur_key] = count

for month in list(month_table_raw.keys()):
    month_table_raw[month] = sorted([(i, j) for i, j in month_table_raw[month].items()],
                                   key = lambda x : x[1], reverse = True)

pre_points_table = {i:give_points(month_table_raw[i], points) for i in month_table_raw.keys()}
    
month_table_dict = month_table_raw.copy()

points_table = {}
points_table_full = {}
        
for cur_month in month_table_raw.keys():
    for cur_tuple in pre_points_table[cur_month]:
        if not cur_month in points_table.keys():
            points_table[cur_month] = {}
            points_table_full[cur_month] = {}
        if cur_tuple[0][0] in points_table[cur_month].keys():
            points_table[cur_month][cur_tuple[0][0]] += cur_tuple[1]
            points_table_full[cur_month][cur_tuple[0][0]].append(cur_tuple[1])
        else:
            points_table[cur_month][cur_tuple[0][0]] = cur_tuple[1]
            points_table_full[cur_month][cur_tuple[0][0]] = [cur_tuple[1]]

points_table = {i:[(j, k) for j, k in points_table[i].items()] for i in month_table_raw.keys()}


final_table = {}
for i in points_table.values():
    for j in i:
        if j[0] in final_table.keys():
            final_table[j[0]] += j[1]
        else:
            final_table[j[0]] = j[1]
final_table = sorted([(i, j) for i, j in final_table.items()],
                     key = lambda x : x[1], reverse = True)
places = give_points(final_table, list(range(1, len(final_table) + 1)))

def get_month_dude_point(month, dude):
    for i in points_table[month]:
        if i[0] == dude:
            return "<b>{1}</b> ({0})".format(", ".join([str(i) for i in
                    sorted(points_table_full[month][dude], reverse = True)]), str(i[1]))

    return "0"

dudes_info = get_certain_users([i[0] for i in final_table])

final_table_df = {"{:02}".format(i):{j[0]: get_month_dude_point(i, j[0]) for j in final_table} 
                  for i in range(1, 12 + 1)}
final_table_df["Place"] = {i[0]:"<b>{0}</b>".format(i[1]) for i in places}
final_table_df["place_int"] = {i[0]:i[1] for i in places}
final_table_df["Sum"] = {i[0]:"<b>{0}</b>".format(i[1]) for i in final_table}
final_table_df["Name"] = {i[0]:"<a class = 'dude-link' href = 'https://vk.com/id{0}'>{1} {2}</a>".format(
    i[0], dudes_info[i[0]]["first_name"], dudes_info[i[0]]["last_name"]) for i in final_table}
final_table_df["Photo"] = {i[0]:"<img class = 'dude-img' src = '{0}'>".format(
    dudes_info[i[0]]["photo_50"]) for i in final_table}

pd.set_option('display.max_colwidth', -1)
output = pd.DataFrame.from_dict(final_table_df).sort_values("place_int").to_html(escape = False,
    columns = ["Place", "Photo", "Name"] + ["{:02}".format(i) for i in range(1, 12+1)] + ["Sum"],
                                                                            index = False)
pd.set_option('display.max_colwidth', 20)
htmlfile = open("threads.html", "w")
htmlfile.write(html_beginning.format("Чемпионат троллей"))
htmlfile.write("<h3>Личный зачёт Троллей-2016</h3>")
htmlfile.write(output)
htmlfile.write(html_ending)
htmlfile.close()

In [14]:
month_table_raw = {}
for comment_raw_dict in comment_dict.values():
    count = None
#     if "likes" in comment_raw_dict.keys():
#         count = post_raw_dict["likes"]["count"]
#     if count is None:
#         continue
    cur_month = comment_raw_dict["date_py"].month
    cur_key = comment_raw_dict["from_id"]
    if cur_key == -info["id"]:
        continue
    if not cur_month in month_table_raw.keys():
        month_table_raw[cur_month] = {}
    if cur_key in month_table_raw[cur_month].keys():
        month_table_raw[cur_month][cur_key] += 1
    else:
        month_table_raw[cur_month][cur_key] = 1

month_table_dict = month_table_raw.copy()
        
for month in list(month_table_raw.keys()):
    month_table_raw[month] = sorted([(i, j) for i, j in month_table_raw[month].items()],
                                   key = lambda x : x[1], reverse = True)

points_table = {i:give_points(month_table_raw[i], points) for i in month_table_raw.keys()}

final_table = {}
for i in points_table.values():
    for j in i:
        if j[0] in final_table.keys():
            final_table[j[0]] += j[1]
        else:
            final_table[j[0]] = j[1]
        
final_table = sorted([(i, j) for i, j in final_table.items()],
                     key = lambda x : x[1], reverse = True)
places = give_points(final_table, list(range(1, len(final_table) + 1)))

def get_month_dude_point(month, dude):
    for i in points_table[month]:
        if i[0] == dude:
            return "<b>{0}</b> ({1})".format(str(i[1]), str(month_table_dict[month][dude]))
    
    if dude in month_table_dict[month].keys():
        return "0 ({0})".format(str(month_table_dict[month][dude]))

    return "0 (0)"

dudes_info = get_certain_users([i[0] for i in final_table])

final_table_df = {"{:02}".format(i):{j[0]: get_month_dude_point(i, j[0]) for j in final_table} 
                  for i in range(1, 12 + 1)}
final_table_df["Place"] = {i[0]:"<b>{0}</b>".format(i[1]) for i in places}
final_table_df["place_int"] = {i[0]:i[1] for i in places}
final_table_df["Sum"] = {i[0]:"<b>{0}</b>".format(i[1]) for i in final_table}
final_table_df["Name"] = {i[0]:"<a class = 'dude-link' href = 'https://vk.com/id{0}'>{1} {2}</a>".format(
    i[0], dudes_info[i[0]]["first_name"], dudes_info[i[0]]["last_name"]) for i in final_table}
final_table_df["Photo"] = {i[0]:"<img class = 'dude-img' src = '{0}'>".format(
    dudes_info[i[0]]["photo_50"]) for i in final_table}

pd.set_option('display.max_colwidth', -1)
output = pd.DataFrame.from_dict(final_table_df).sort_values("place_int").to_html(escape = False,
    columns = ["Place", "Photo", "Name"] + ["{:02}".format(i) for i in range(1, 12+1)] + ["Sum"],
                                                                            index = False)
pd.set_option('display.max_colwidth', 20)
htmlfile = open("comments.html", "w", encoding = "utf-8")
htmlfile.write(html_beginning.format("Чемпионат флудеров"))
htmlfile.write("<h3>Личный зачёт Флудеров-2016</h3>")
htmlfile.write(output)
htmlfile.write(html_ending)
htmlfile.close()

In [20]:
month_table_raw = {}

max_likes = 0
max_likes_dict = None

for post_raw_dict in posts_dict.values():
    count = None
    if "likes" in post_raw_dict.keys():
        count = post_raw_dict["likes"]["count"]
    if count is None:
        continue

    cur_month = post_raw_dict["date_py"].month
    cur_key = post_raw_dict["from_id"]
    if cur_key == -info["id"]:
        continue
        
    if count > max_likes:
        max_likes = count
        max_likes_dict = post_raw_dict
        
    if not cur_month in month_table_raw.keys():
        month_table_raw[cur_month] = {}
    if cur_key in month_table_raw[cur_month].keys():
        month_table_raw[cur_month][cur_key] += count
    else:
        month_table_raw[cur_month][cur_key] = count

        
month_table_dict = month_table_raw.copy()
        
for month in list(month_table_raw.keys()):
    month_table_raw[month] = sorted([(i, j) for i, j in month_table_raw[month].items()],
                                   key = lambda x : x[1], reverse = True)

points_table = {i:give_points(month_table_raw[i], points) for i in month_table_raw.keys()}

final_table = {}
for i in points_table.values():
    for j in i:
        if j[0] in final_table.keys():
            final_table[j[0]] += j[1]
        else:
            final_table[j[0]] = j[1]
        
final_table = sorted([(i, j) for i, j in final_table.items()],
                     key = lambda x : x[1], reverse = True)
places = give_points(final_table, list(range(1, len(final_table) + 1)))

def get_month_dude_point(month, dude):
    for i in points_table[month]:
        if i[0] == dude:
            return "<b>{0}</b> ({1})".format(str(i[1]), str(month_table_dict[month][dude]))
    
    if dude in month_table_dict[month].keys():
        return "0 ({0})".format(str(month_table_dict[month][dude]))

    return "0 (0)"

dudes_info = get_certain_users([i[0] for i in final_table])

final_table_df = {"{:02}".format(i):{j[0]: get_month_dude_point(i, j[0]) for j in final_table} 
                  for i in range(1, 12 + 1)}
final_table_df["Place"] = {i[0]:"<b>{0}</b>".format(i[1]) for i in places}
final_table_df["place_int"] = {i[0]:i[1] for i in places}
final_table_df["Sum"] = {i[0]:"<b>{0}</b>".format(i[1]) for i in final_table}
final_table_df["Name"] = {i[0]:"<a class = 'dude-link' href = 'https://vk.com/id{0}'>{1} {2}</a>".format(
    i[0], dudes_info[i[0]]["first_name"], dudes_info[i[0]]["last_name"]) for i in final_table}
final_table_df["Photo"] = {i[0]:"<img class = 'dude-img' src = '{0}'>".format(
    dudes_info[i[0]]["photo_50"]) for i in final_table}

pd.set_option('display.max_colwidth', -1)
output = pd.DataFrame.from_dict(final_table_df).sort_values("place_int").to_html(escape = False,
    columns = ["Place", "Photo", "Name"] + ["{:02}".format(i) for i in range(1, 12+1)] + ["Sum"],
                                                                            index = False)
pd.set_option('display.max_colwidth', 20)
htmlfile = open("likes_posts.html", "w", encoding = "utf-8")
htmlfile.write(html_beginning.format("Чемпионат постящих няшек"))
htmlfile.write("<h3>Приз Зрительских Симпатий: Посты-2016</h3>")
htmlfile.write(output)
htmlfile.write(html_ending)
htmlfile.close()

print(max_likes_dict)

{'likes': {'count': 86}, 'date': 1464438415, 'comments': {'count': 10}, 'date_py': datetime.datetime(2016, 5, 28, 15, 26, 55), 'owner_id': -647, 'from_id': 153107746, 'attachments': [{'type': 'photo', 'photo': {'access_key': '676a00cd24b287c73f', 'date': 1464438415, 'photo_130': 'https://pp.vk.me/c631417/v631417746/3248b/rJmbpcWk3A0.jpg', 'width': 960, 'user_id': 153107746, 'photo_807': 'https://pp.vk.me/c631417/v631417746/3248d/0b26VxKfBEM.jpg', 'photo_604': 'https://pp.vk.me/c631417/v631417746/3248c/7XSood9crUo.jpg', 'post_id': 565194, 'photo_75': 'https://pp.vk.me/c631417/v631417746/3248a/bwtJPpZnwcc.jpg', 'owner_id': -647, 'height': 601, 'album_id': -8, 'id': 416891608, 'text': '', 'photo_1280': 'https://pp.vk.me/c631417/v631417746/3248e/VrC0f_a7TVQ.jpg'}}], 'id': 565194, 'post_type': 'post', 'text': 'Радости пост (автор картинки - респек тебе)', 'reposts': {'count': 1}}


In [22]:
month_table_raw = {}

max_likes = 0
max_likes_dict = None

for comment_raw_dict in comment_dict.values():
    count = None
    if "likes" in comment_raw_dict.keys():
        count = comment_raw_dict["likes"]["count"]
    if count is None:
        continue
    cur_month = comment_raw_dict["date_py"].month
    cur_key = comment_raw_dict["from_id"]
    if cur_key == -info["id"]:
        continue
        
    if count > max_likes:
        max_likes = count
        max_likes_dict = comment_raw_dict
        
        
    if not cur_month in month_table_raw.keys():
        month_table_raw[cur_month] = {}
    if cur_key in month_table_raw[cur_month].keys():
        month_table_raw[cur_month][cur_key] += count
    else:
        month_table_raw[cur_month][cur_key] = count
        
month_table_dict = month_table_raw.copy()
        
for month in list(month_table_raw.keys()):
    month_table_raw[month] = sorted([(i, j) for i, j in month_table_raw[month].items()],
                                   key = lambda x : x[1], reverse = True)

points_table = {i:give_points(month_table_raw[i], points) for i in month_table_raw.keys()}

final_table = {}
for i in points_table.values():
    for j in i:
        if j[0] in final_table.keys():
            final_table[j[0]] += j[1]
        else:
            final_table[j[0]] = j[1]
        
final_table = sorted([(i, j) for i, j in final_table.items()],
                     key = lambda x : x[1], reverse = True)
places = give_points(final_table, list(range(1, len(final_table) + 1)))

def get_month_dude_point(month, dude):
    for i in points_table[month]:
        if i[0] == dude:
            return "<b>{0}</b> ({1})".format(str(i[1]), str(month_table_dict[month][dude]))
    
    if dude in month_table_dict[month].keys():
        return "0 ({0})".format(str(month_table_dict[month][dude]))

    return "0 (0)"

dudes_info = get_certain_users([i[0] for i in final_table])

final_table_df = {"{:02}".format(i):{j[0]: get_month_dude_point(i, j[0]) for j in final_table} 
                  for i in range(1, 12 + 1)}
final_table_df["Place"] = {i[0]:"<b>{0}</b>".format(i[1]) for i in places}
final_table_df["place_int"] = {i[0]:i[1] for i in places}
final_table_df["Sum"] = {i[0]:"<b>{0}</b>".format(i[1]) for i in final_table}
final_table_df["Name"] = {i[0]:"<a class = 'dude-link' href = 'https://vk.com/id{0}'>{1} {2}</a>".format(
    i[0], dudes_info[i[0]]["first_name"], dudes_info[i[0]]["last_name"]) for i in final_table}
final_table_df["Photo"] = {i[0]:"<img class = 'dude-img' src = '{0}'>".format(
    dudes_info[i[0]]["photo_50"]) for i in final_table}

pd.set_option('display.max_colwidth', -1)
output = pd.DataFrame.from_dict(final_table_df).sort_values("place_int").to_html(escape = False,
    columns = ["Place", "Photo", "Name"] + ["{:02}".format(i) for i in range(1, 12+1)] + ["Sum"],
                                                                            index = False)
pd.set_option('display.max_colwidth', 20)
htmlfile = open("likes_comments.html", "w", encoding = "utf-8")
htmlfile.write(html_beginning.format("Чемпионат комментирующих няшек"))
htmlfile.write("<h3>Приз Зрительских Симпатий: Комментарии-2016</h3>")
htmlfile.write(output)
htmlfile.write(html_ending)
htmlfile.close()

print(max_likes_dict)

{'likes': {'count': 102}, 'date': 1462572425, 'date_py': datetime.datetime(2016, 5, 7, 1, 7, 5), 'from_id': 9739479, 'id': 559192, 'text': 'Как же это всё произошло, \nВедь так мало времени прошло, \nМне понизил должность тип один \nИ у него на это пять причин. \n\nПервая причина — Йоса план, \nА вторая — Феттеля таран, \nТретья это очень юный Макс, \nВзял да и объехал на раз. \nЧетвёртая причина это Дэн —\nЕго квалификационный ( =D ) плен.\nА пятая причина это Я,\nКвят в Ред Булле, ля ля ля ля ля.'}
